In [1]:
import pandas as pd
import paramiko
import datetime
import os
from haversine import haversine
import numpy as np
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
os.getcwd()

'/home/jian/Projects/C21/Calculate_Dist_Joann_0318'

In [2]:
df=pd.read_table("/home/jian/Projects/C21/Calculate_Dist_Joann_0318/Customer_ID_Master_File_03042019.txt.xz",
                 compression="xz",sep="|",dtype=str)
df.shape

(4201416, 10)

In [3]:
df_4_stores=pd.DataFrame()
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"DOWNTOWN","Google_Address":"22 Cortlandt St, New York, NY 10007","Googel_Coor":[[40.7104649,-74.0107144]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"REGO_PARK","Google_Address":"61-35 Junction Blvd, Rego Park, NY 11374","Googel_Coor":[[40.7332205,-73.8631858]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"CROSS_COUNTY","Google_Address":"750 Central Park Ave, Yonkers, NY 10704","Googel_Coor":[[40.9262461,-73.8553119]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Googel_Coor":[[40.9150771,-74.0579318]]}))
'''
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Googel_Coor":[[40.9150771,-74.0579318]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Googel_Coor":[[40.9150771,-74.0579318]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Googel_Coor":[[40.9150771,-74.0579318]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Googel_Coor":[[40.9150771,-74.0579318]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Googel_Coor":[[40.9150771,-74.0579318]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Googel_Coor":[[40.9150771,-74.0579318]]}))
'''

df_4_stores=df_4_stores.reset_index()
del df_4_stores['index']

In [4]:
df_customer_zips=df[['Customer_zip_code']]
df_customer_zips=df_customer_zips[~pd.isnull(df_customer_zips['Customer_zip_code'])]
print(df_customer_zips.shape)
df_customer_zips['Cleaned_zip_cd']=df_customer_zips['Customer_zip_code'].apply(lambda x: x.split("-")[0].split(" ")[0])
df_customer_zips['Cleaned_zip_cd']=df_customer_zips['Cleaned_zip_cd'].apply(lambda x: x.zfill(5)[:5])
# df_unique_zip=df_unique_zip.drop_duplicates()



(3492554, 1)


In [5]:
df_customer_zips['digital_1']=df_customer_zips['Cleaned_zip_cd'].apply(lambda x: x[0])
df_customer_zips['digital_2']=df_customer_zips['Cleaned_zip_cd'].apply(lambda x: x[1])
df_customer_zips['digital_3']=df_customer_zips['Cleaned_zip_cd'].apply(lambda x: x[2])
df_customer_zips['digital_4']=df_customer_zips['Cleaned_zip_cd'].apply(lambda x: x[3])
df_customer_zips['digital_5']=df_customer_zips['Cleaned_zip_cd'].apply(lambda x: x[4])

numb_0_9=[str(x) for x in range(10)]

df_customer_zips[(df_customer_zips['digital_1'].isin(numb_0_9)) &\
              (df_customer_zips['digital_2'].isin(numb_0_9)) &\
             (df_customer_zips['digital_3'].isin(numb_0_9)) &\
             (df_customer_zips['digital_4'].isin(numb_0_9)) &\
             (df_customer_zips['digital_5'].isin(numb_0_9))].shape

(3484115, 7)

In [6]:
df_unique_zips=df_customer_zips[['Cleaned_zip_cd']].drop_duplicates()

for index, row in df_4_stores.iterrows():
    store_name=row['Store_Name']
    store_coor=row['Googel_Coor']
    df_unique_zips[store_name+"_Google_Location"]=str(store_coor)
    df_unique_zips[store_name+"_Distance_Miles"]=np.nan

In [7]:
for index, row in df_unique_zips.iterrows():
    customer_zip=row['Cleaned_zip_cd']
    if customer_zip in zip_centers.keys():
        df_unique_zips.loc[index,"DOWNTOWN_Distance_Miles"]=haversine(zip_centers[customer_zip],[40.7104649,-74.0107144],miles=True)
        df_unique_zips.loc[index,"REGO_PARK_Distance_Miles"]=haversine(zip_centers[customer_zip],[40.7332205, -73.8631858],miles=True)
        df_unique_zips.loc[index,"CROSS_COUNTY_Distance_Miles"]=haversine(zip_centers[customer_zip],[40.9262461, -73.8553119],miles=True)
        df_unique_zips.loc[index,"PARAMUS_Distance_Miles"]=haversine(zip_centers[customer_zip],[40.9150771, -74.0579318],miles=True)
    

In [8]:
df_customer_zips=df_customer_zips[['Customer_zip_code','Cleaned_zip_cd']].drop_duplicates()
df_customer_zips=pd.merge(df_customer_zips,df_unique_zips,on="Cleaned_zip_cd",how="left")

In [9]:
output=pd.merge(df,df_customer_zips,on="Customer_zip_code",how="left")

In [10]:
'''
del output['Cleaned_zip_cd']
del output['DOWNTOWN_Google_Location']
del output['REGO_PARK_Google_Location']
del output['CROSS_COUNTY_Google_Location']
del output['PARAMUS_Google_Location']
'''

"\ndel output['Cleaned_zip_cd']\ndel output['DOWNTOWN_Google_Location']\ndel output['REGO_PARK_Google_Location']\ndel output['CROSS_COUNTY_Google_Location']\ndel output['PARAMUS_Google_Location']\n"

In [11]:
output.shape

(4201416, 19)

In [12]:
output[pd.isnull(output['PARAMUS_Distance_Miles'])].shape

(965364, 19)

In [13]:
output.to_csv("/home/jian/Projects/C21/Calculate_Dist_Joann_0318/Customer_ID_Master_File_03042019_with_dist_JL_20180401_with_extra_cols.txt.xz",
             sep="|",compression="xz")

In [7]:
read_output=pd.read_table("/home/jian/Projects/C21/Calculate_Dist_Joann_0318/Customer_ID_Master_File_03042019_with_dist_JL_20180321.txt.xz",
             sep="|",compression="xz",dtype=str)


,Unnamed: 0,Customer_id,Email_address_hash,Sign_up_date,Sign_up_channel,Customer_zip_code,Purchase_channel,Email_unsubscribe_indicator,Email_undeliverable_indicator,Transaction_count_to_date,Transaction_amount_to_ date,DOWNTOWN_Distance_Miles,REGO_PARK_Distance_Miles,CROSS_COUNTY_Distance_Miles,PARAMUS_Distance_Miles
0,0,3083952,9fea52f236a80bb4305a911da039d7cbe7c6433f97387e...,09-DEC-12,POS,07057,STORE,N,Valid,244,16606.27 ...,11.214633284766451,15.284176505391745,13.999744547091291,4.827852460047667
1,1,8733785,7ca5f57523b5f7d38409493a6b06adf11c654c6deb5d3f...,11-SEP-14,POS,07960,STORE,N,Valid,64,3417.27 ...,25.407028819269403,32.69201383368275,33.8952595373588,23.555941204272102


In [8]:
read_output.shape

(4201416, 15)

In [6]:
read_output[read_output['Email_address_hash']=="724c440ec9c49b8843d352c4faf1f9daad86a1236cb023d2b9f85b66b8e72a37"]



,Unnamed: 0,Customer_id,Email_address_hash,Sign_up_date,Sign_up_channel,Customer_zip_code,Purchase_channel,Email_unsubscribe_indicator,Email_undeliverable_indicator,Transaction_count_to_date,Transaction_amount_to_ date,DOWNTOWN_Distance_Miles,REGO_PARK_Distance_Miles,CROSS_COUNTY_Distance_Miles,PARAMUS_Distance_Miles
1039145,1039145,15642619,724c440ec9c49b8843d352c4faf1f9daad86a1236cb023...,21-NOV-17,Weblinc,11237,ONLINE,N,Valid,1,135.94 ...,4.851576775227861,3.632266095876557,15.85901222377715,16.434567773349762
3892943,3892943,15877663,724c440ec9c49b8843d352c4faf1f9daad86a1236cb023...,23-DEC-17,POS,96821,STORE,Y,Valid,1,-57.97 ...,4954.845777709682,4962.0334930217405,4959.488441958884,4949.33722729779
4123938,4123938,15369160,724c440ec9c49b8843d352c4faf1f9daad86a1236cb023...,09-OCT-17,ereceipt,NaN,STORE,Y,Valid,1,12.99 ...,NaN,NaN,NaN,NaN


In [ ]:
test=read_output[pd.isnull(read_output[''])]

In [9]:
read_output=pd.read_table("/home/jian/Projects/C21/Calculate_Dist_Joann_0318/Customer_ID_Master_File_03042019_with_dist_JL_20180401_with_extra_cols.txt.xz",
             sep="|",compression="xz",dtype=str)

zip_mapping=read_output[['Customer_zip_code','Cleaned_zip_cd']].drop_duplicates()
zip_mapping.to_csv("/home/jian/Projects/C21/Calculate_Dist_Joann_0318/Customer_ID_Master_File_unique_zips_mapping.txt",index=False,sep="|")

In [6]:
read_output.shape

(4201416, 20)

In [7]:
len(read_output['Email_address_hash'].unique())

3830332

In [8]:
len(read_output['Customer_id'].unique())

3978957